<a href="https://colab.research.google.com/github/mudasirhaider007/penetration-testing/blob/main/penetration_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
Simple Service Enumerator (Beginner-friendly, safe)

Usage:
    python simple_enum.py            # scans localhost
    python simple_enum.py 192.168.56.101  # scan one host
"""

import platform, subprocess, socket, sys, json, threading, time
from queue import Queue
from datetime import datetime
from http.client import HTTPConnection, HTTPSConnection

COMMON_PORTS = [21, 22, 23, 25, 53, 80, 110, 123, 143, 443, 587, 3389]

PING_TIMEOUT = 1
CONNECT_TIMEOUT = 1.0
THREADS = 20

def is_alive(host: str) -> bool:
    """Ping the host once (system ping). Returns True if ping succeeds."""
    param = '-n' if platform.system().lower() == 'windows' else '-c'
    try:
        completed = subprocess.run(['ping', param, '1', host],
                                   stdout=subprocess.DEVNULL,
                                   stderr=subprocess.DEVNULL,
                                   timeout=PING_TIMEOUT + 1)
        return completed.returncode == 0
    except Exception:
        return False

def tcp_connect(host: str, port: int, timeout: float = CONNECT_TIMEOUT) -> bool:
    """Attempt TCP connect. Returns True if the port is open (connect succeeds)."""
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.settimeout(timeout)
            return s.connect_ex((host, port)) == 0
    except Exception:
        return False

def grab_banner(host: str, port: int, timeout: float = CONNECT_TIMEOUT) -> str:
    """Try a short read from the socket to capture a banner. Non-blocking-ish."""
    try:
        with socket.socket() as s:
            s.settimeout(timeout)
            s.connect((host, port))
            try:
                data = s.recv(1024)
                return data.decode(errors='ignore').strip()
            except Exception:
                return ''
    except Exception:
        return ''

def http_probe(host: str, port: int, use_https: bool = False) -> dict:
    """If HTTP(S) port, perform a simple GET and return status and headers (no body dump)."""
    try:
        if use_https:
            conn = HTTPSConnection(host, port=port, timeout=2)
        else:
            conn = HTTPConnection(host, port=port, timeout=2)
        conn.request("GET", "/")
        resp = conn.getresponse()
        headers = dict(resp.getheaders())
        snippet = resp.read(512)  # read up to 512 bytes
        conn.close()
        return {'status': resp.status, 'reason': resp.reason, 'headers': headers, 'snippet': snippet.decode(errors='ignore')[:200]}
    except Exception as e:
        return {'error': str(e)}

def worker(host: str, port_queue: Queue, results: list, lock: threading.Lock):
    while True:
        try:
            port = port_queue.get_nowait()
        except Exception:
            return
        open_flag = tcp_connect(host, port)
        entry = {'port': port, 'open': open_flag}
        if open_flag:
            entry['banner'] = grab_banner(host, port)
            if port in (80, 8080):
                entry['http'] = http_probe(host, port, use_https=False)
            elif port in (443, 8443):
                entry['http'] = http_probe(host, port, use_https=True)
        with lock:
            results.append(entry)
        port_queue.task_done()

def scan_host(host: str, ports: list):
    host_result = {'host': host, 'alive': False, 'ports': []}
    if not is_alive(host):
        host_result['alive'] = False
        return host_result
    host_result['alive'] = True

    q = Queue()
    for p in ports:
        q.put(p)
    results = []
    lock = threading.Lock()
    threads = []
    for _ in range(min(THREADS, max(1, q.qsize()))):
        t = threading.Thread(target=worker, args=(host, q, results, lock), daemon=True)
        t.start()
        threads.append(t)
    q.join()
    host_result['ports'] = sorted(results, key=lambda x: x['port'])
    return host_result

def main():
    if len(sys.argv) > 1:
        hosts = [sys.argv[1]]
    else:
        hosts = ['127.0.0.1']  # default: localhost for safe tests
    ports = COMMON_PORTS

    all_results = {'scanned_at': datetime.utcnow().isoformat() + 'Z', 'hosts': []}
    for h in hosts:
        print(f"[+] Scanning {h} ...")
        res = scan_host(h, ports)
        all_results['hosts'].append(res)
        print(f"    alive={res['alive']}, open_ports={[p['port'] for p in res['ports'] if p['open']]}")
    fname = f"enum_results_{int(time.time())}.json"
    with open(fname, 'w') as f:
        json.dump(all_results, f, indent=2)
    print(f"[+] Results saved to {fname}")

if __name__ == "__main__":
    main()


[+] Scanning -f ...
    alive=False, open_ports=[]
[+] Results saved to enum_results_1757744928.json


/tmp/ipython-input-3736503506.py:118: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  all_results = {'scanned_at': datetime.utcnow().isoformat() + 'Z', 'hosts': []}
